# Notebook 1. Processing Pipeline Exploration
### By Max Pechyonkin
---
## Introduction

> <strong><font color=red>Note:</font></strong>
I understand that you might have limited time. So, if you would rather read only one notebook, rather than all three, please only read the last one, called `02_data_pipeline_refactored.ipynb`, it has the final result. The first two notebooks contained information about how I arrived at that result.

This is an exploratory notebook, where I go step by step and figure out how to deal with this or that feature. While reading this notebook, please also look at the accompanying hand-written digital notes in the PDF format that I submitted with the project. They provide some clarity about why I made certain decisions. 

This is notebook is not production-ready. The next notebook performs refactoring and data processing in one simple API call: `datacleaner.transform(X)`, which uses all the steps that I devised while working on the data in this notebook.

# 1. Read in CSV, ignore invalid rows

In [ ]:
import pandas as pd
import json

DATA_PATH = '7004_1.csv'

df = pd.read_csv(DATA_PATH, error_bad_lines=False, warn_bad_lines=False)

In [ ]:
df.shape

In [ ]:
df.columns

# 2. Remove rows with all NaNs

In [ ]:
all_nan_cols = df.isnull().all()
all_nan_cols[all_nan_cols]

all_nan_col_names = all_nan_cols[all_nan_cols].index
all_nan_col_names

df = df.drop(all_nan_col_names, axis=1)

# 3. Remove rows with NaNs percentage > 99%

In [ ]:
def get_nans_percentages(data):
    """
    Get percentages of NaNs for each column.
    
    Returns Series with index of column names and float values.
    """
    nrows, _ = data.shape
    return (data.isnull().sum(axis=0) / nrows).sort_values(ascending=False)

def get_colnames_with_nans_above_thresh(data, thresh: float):
    """
    Return column names with NaNs proportions above the given level.
    """
    percentages = get_nans_percentages(data)
    result = percentages[percentages > thresh]
    return result.index



In [ ]:
colnames_with_too_many_nans = get_colnames_with_nans_above_thresh(df, thresh=0.99)
colnames_with_too_many_nans

df = df.drop(colnames_with_too_many_nans, axis=1)

In [ ]:
df.head(10)

# 4. Remove marked columns 🔴

Analysis why columns below need to be removed is in the notes PDF accompanying this submission.

Main reasons are:

- features that are `id`-like, i.e. we don't want to train for a specific product id, this will overfit
- features that I deemed not very relevant - such as dates, even though I could be possible to turn those dates into categorical features like day of the week, holiday, season etc. For now I decided to not put additional effort in there and focus on building a working MVP.
- features having NLP data, such as reviews, name of the product. I could build an NLP feature exctractor based on a neural net that could probably work, but to save time I ignore it for now
- URLs to images and webpages. Some of them are not available, or extracting requires additional work that is not feasible given my time constraints for this project.

In [ ]:
for cname in df.columns:
    print(f"    '{cname}',")

In [ ]:
cols_to_remove = [
    'id',
    'asins',
    'dateAdded',
    'dateUpdated',
    'descriptions',
    'ean',
    'features',
    'imageURLs',
    'keys',
    'manufacturerNumber',
    'merchants',
    'name',
    'prices.amountMin',
    'prices.dateAdded',
    'prices.dateSeen',
    'prices.sourceURLs',
    'skus',
    'sourceURLs',
    'upc',
    'weight',
]

df = df.drop(cols_to_remove, axis=1)

In [ ]:
df

## Dealing with currency and price conversion

In [ ]:
df['prices.currency'].value_counts()


First observation - there appear to be some illegal currencies. Second is that there are not only prices in USD, but also in AUD, CAD, EUR and GBP. I could discard that, but I will convert all prices to USD. The best way to do it would be to retrieve the exchange rates for when the prices were published, but I will simplify and take the current rates. In production, this would definitely require more careful analysis. 

Another question I would ask is whether features are good predictors in other markets that use other currencies. It might be the case that for a given local market it is better to have a separate model that learn important relationships in the local market.

### Currency, Part 1: Remove rows that are not currencies.

In [ ]:
currencies_all = df['prices.currency'].value_counts()
currencies_all

In [ ]:
legal_currencies = currencies_all[currencies_all > 20 ].index
legal_currencies

In [ ]:
'SHOEBACCA LTD. - Walmart.com' not in legal_currencies

In [ ]:
rows_illegal_currency = df['prices.currency'].apply(lambda x: x not in legal_currencies)
rows_illegal_fx = rows_illegal_currency[rows_illegal_currency == True].index
rows_illegal_fx

In [ ]:
df.shape

In [ ]:
df = df.drop(rows_illegal_fx, axis=0)

In [ ]:
df.shape

In [ ]:
df['prices.currency'].value_counts()

### Currency, Part 2: calculate the price based on currency and FX rate

- `fx_rates` is a mapping from dictionary to a scalar that will be multiply price in currency to make it price in dollars
- I used Google to get the rates

In [ ]:
fx_rates = {
    'USD': 1.00,
    'AUD': 0.68,
    'CAD': 0.75,
    'EUR': 1.10,
    'GBP': 1.25,
}

In [ ]:
df['prices.amountMax'].isnull().sum()

In [ ]:
df['prices.currency'].isnull().sum()

In [ ]:
def is_invalid_price_string(s):
    s = str(s)
    illegal_chars = '-T:Z'
    for c in illegal_chars:
        if c in s:
            return True
    return False

illegal_price_mask = df['prices.amountMax'].apply(is_invalid_price_string)
illegal_price_idxs = illegal_price_mask[illegal_price_mask].index
illegal_price_idxs

As you can see below, these rows clearly don't belong here. Not only the price is illegal, it looks like these are not even shoes.

In [ ]:
df.loc[illegal_price_idxs]

Let's remove these rows.

In [ ]:
df = df.drop(illegal_price_idxs)

In [ ]:
def convert_to_usd(row):
    price = float(row['prices.amountMax'])
    currency = row['prices.currency']
    if not currency:
        # if currency is NaN assume USD and return price
        return price
    fx_rate = fx_rates[currency]
    return price * fx_rate

df['price'] = df.apply(convert_to_usd, axis=1)
df = df.drop(['prices.amountMax'], axis=1)
df

## Cleaning `isSale` column

In [ ]:
def get_unique_nans(colname, data, display=True):
    uniques = data[colname].unique()
    nans = data[colname].isnull().sum()
    print(uniques)
    print(f'Uniques: {len(uniques)}')
    print(f'Nans: {nans}')
    if not display:
        return uniques, nans

In [ ]:
get_unique_nans('prices.isSale', df)

In [ ]:
def process_is_sale(x):
    if isinstance(x, bool):
        return x
    elif x.capitalize() == 'True':
        return True
    elif x.capitalize() == 'False':
        return False
    else:
        print(x)
        print(type(x))
        raise ValueError("Something went wrong!")

df['prices.isSale'] = df['prices.isSale'].apply(process_is_sale)

In [ ]:
df['prices.isSale']

## Cleaning `categories` column

Since categories are presented as comma separated values, let's see how many unique categories are there by splitting them.

After splitting, I realized that the categoriez are not very informative and decided to remove the column.

In [ ]:
all_categories = []

for string in df.categories.unique():
    string = string.lower()
    if 'shoes' not in string:
        print(string)
    subcategories = string.split(',')
    subcategories = [s.strip() for s in subcategories]
    for s in subcategories:
        if s not in all_categories:
            all_categories.append(s)
            
print(len(all_categories))

In [ ]:
all_categories

In [ ]:
df = df.drop(['categories'], axis=1)

## Cleaning `brand` column

This one is ready to be categorized.

In [ ]:
get_unique_nans('brand', df)

## Cleaning `prices.merchant`

Ready for categorization.

In [ ]:
get_unique_nans('prices.merchant', df)

## Cleaning `prices.condition` column

In [ ]:
get_unique_nans('prices.condition', df)

## Cleaning `colors` column

This is fairly complex column, with a lot of messy colors. I will simply make it lower-case and remove dashes for the time being.

In [ ]:
unique_colors, nans_color = get_unique_nans('colors', df, display=False)
unique_colors, nans_color

In [ ]:
df['colors'] = df['colors'].apply(lambda x: x.lower().replace(' ', '').replace('-', '') if not pd.isnull(x) else x)

## Cleaning `manufacturer` column
This one is ready for encoding as a categorical feature.

In [ ]:
get_unique_nans('manufacturer', df)

## Cleaning `prices.offer` column

I will leave it as is.

In [ ]:
get_unique_nans('prices.offer', df)

## Cleaning `prices.shipping` column

I will broadly categorize this variable into "Free" vs "Not Free"

In [ ]:
get_unique_nans('prices.shipping', df)

In [ ]:
def process_shipping(x):
    if pd.isnull(x):
        return x
    
    x = x.lower()
    if 'free' in x:
        return 'free'
    else:
        return 'not free'
    return x
    
    
df['prices.shipping'] = df['prices.shipping'].apply(process_shipping)
df['prices.shipping'].value_counts()

## Cleaning `sizes` column

To simplify, I will calculate how many sizes are available for a particulate shoe.

In [ ]:
get_unique_nans('sizes', df)

In [ ]:
def process_sizes(x):
    if pd.isnull(x):
        return x
    return len(x.split(','))

df['sizes'] = df['sizes'].apply(process_sizes)
df['sizes'].value_counts()

## Cleaning `dimension` column

Let's see if dimension can be easily processed.

To simplify, I will create a dimension number as the sum of all 3 dimensions. This will help to deal with inconsistencies of the order of the three dimensions (e.g. is it height-width-depth or depth-heitht-width or some other combination).

In [ ]:
unique_dims, _ = get_unique_nans('dimension', df, display=False)

In [ ]:
for d in unique_dims:
    if not pd.isnull(d) and len(d.split('x')) != 3:
        print(d)

This means almost all dimensions can be split by `x` except for '32 inches' one.

In [ ]:
def process_dimensions(x):
    if pd.isnull(x):
        return x
    
    # deal with edge case
    if x == '32 inches':
        return 32.0
    
    x = x.lower()
    dims = x.split('x')
    dims = [d.replace(' ', '').replace('in', '') for d in dims]
    dims = map(float, dims)
    return sum(dims)
    
    
df['dimension'] = df['dimension'].apply(process_dimensions)
df['dimension']

## Cleaning `prices.size` column

Drop this column, since too few values and mostly very different.

In [ ]:
get_unique_nans('prices.size', df)

In [ ]:
df['prices.size'].value_counts()

In [ ]:
df = df.drop('prices.size', axis=1)

## Cleaning `prices.color` column

Drop this too, for same reason.

In [ ]:
get_unique_nans('prices.color', df)

In [ ]:
df = df.drop('prices.color', axis=1)

## Cleaning `prices.returnPolicy` column

Ready for categorizing.

In [ ]:
get_unique_nans('prices.returnPolicy', df)

In [ ]:
df['prices.returnPolicy'].value_counts()

## Calculating Average Review Score

I decided to calculate average rating for reviews with rating. If there is no rating, return zero.

In [ ]:
no_reviews_idxs = df['reviews'].isnull()

In [ ]:
df['reviews'][~no_reviews_idxs]

In [ ]:
def parse_review(x):
    if pd.isnull(x):
        return x

    rating = 0.0
    try:
        reviews = json.loads(x)
        for r in reviews:
            if 'rating' in r.keys():
                rating += float(r['rating'])
        return rating / len(reviews)
    except:
        return rating

df['reviews'] = df['reviews'].apply(parse_review)
df['reviews'].value_counts()

## Examining and splitting the data

In [ ]:
df

## Get rid of price outliers

In [ ]:
df = df.drop(df[df.price > 2000].index)
df

In [ ]:
X, y = df.drop('price', axis=1), df['price']

In [ ]:
X

In [ ]:
y

In [ ]:
X.describe()

In [ ]:
X.columns

In [ ]:
categorical=[
    'brand',
    'colors',
    'manufacturer',
    'prices.condition',
    'prices.currency',
    'prices.isSale',
    'prices.merchant',
    'prices.offer',
    'prices.returnPolicy',
    'prices.shipping',
]

In [ ]:
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# apply random permutation to the data (required by CatBoost)
perm = np.random.permutation(len(X))
X = X.iloc[perm].reset_index(drop=True)
y = y.iloc[perm].reset_index(drop=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
ce_cat = ce.CatBoostEncoder(cols=categorical)

In [ ]:
X_train

In [ ]:
X_train = ce_cat.fit_transform(X_train, y_train)
X_train

In [ ]:
def logify(data):
    return np.log(1 + data)

def unlogify(data):
    return np.exp(data) - 1

In [ ]:
y_train = logify(y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
model = GradientBoostingRegressor(max_leaf_nodes=None, n_estimators=50)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
X_test

In [ ]:
X_test_transformed = ce_cat.transform(X_test)
X_test_transformed

In [ ]:
X.describe().columns

In [ ]:
cols_to_fill_nas = ['dimension', 'reviews', 'sizes']
for c in cols_to_fill_nas:
    X_test_transformed[c] = X_test_transformed[c].fillna(-999.9)
X_test_transformed

In [ ]:
preds = model.predict(X_test_transformed)

In [ ]:
mean_squared_error(preds, logify(y_test))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(preds, logify(y_test))

In [ ]:
plt.scatter(unlogify(preds), y_test)

In [ ]:
plt.hist(np.log(1+y_train), bins=30)

In [ ]:
lg = np.log(1+y_train)
un = np.exp(lg) - 1

In [ ]:
plt.hist(un, bins=30)